In [1]:
!pip install transformers datasets scikit-learn langgraph --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.4/152.4 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.0/50.0 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.5/216.5 kB 16.3 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
import torch
import os
from datetime import datetime
from datasets import Dataset
from transformers import (
    DistilBertTokenizerFast,
    DistilBertForSequenceClassification,
    TrainingArguments,
    Trainer,
    pipeline
)
from sklearn.metrics import f1_score
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import MemorySaver
from concurrent.futures import ThreadPoolExecutor
import numpy as np

from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import MemorySaver
from concurrent.futures import ThreadPoolExecutor
from langgraph.checkpoint.memory import MemorySaver
import logging



In [4]:


# Setup logging
log_file = 'classification_log.tsv'
open(log_file, "a").close()  # Ensure file exists
logger = logging.getLogger()
logger.setLevel(logging.INFO)
if logger.hasHandlers():
    logger.handlers.clear()
file_handler = logging.FileHandler(log_file)
formatter = logging.Formatter('%(asctime)s\t%(message)s')
file_handler.setFormatter(formatter)
logger.addHandler(file_handler)

NUM_LABELS = 28
TEMP_DATA_FILE = "for_update_data.tsv"
model_name = "distilbert-base-uncased"
emotion_labels = [
    "admiration", "amusement", "anger", "annoyance", "approval", "caring", "confusion",
    "curiosity", "desire", "disappointment", "disapproval", "disgust", "embarrassment",
    "excitement", "fear", "gratitude", "grief", "joy", "love", "nervousness", "optimism",
    "pride", "realization", "relief", "remorse", "sadness", "surprise", "neutral"
]

# Load Dataset
train_df = pd.read_csv('/content/train.tsv', sep='\t', header=None)
dev_df = pd.read_csv('/content/dev.tsv', sep='\t', header=None)
test_df = pd.read_csv('/content/test.tsv', sep='\t', header=None)

def convert_to_multihot(df, num_labels=NUM_LABELS):
    def encode(label_str):
        if isinstance(label_str, str):
            label_indices = list(map(int, label_str.split(',')))
        else:
            label_indices = []
        multihot = np.zeros(num_labels, dtype=np.float32)
        for idx in label_indices:
            if 0 <= idx < num_labels:
                multihot[idx] = 1.0
        return multihot
    df = df.rename(columns={0: 'text', 1: 'label'})
    df['label'] = df['label'].apply(encode)
    return df

train_df = convert_to_multihot(train_df)
dev_df = convert_to_multihot(dev_df)
test_df = convert_to_multihot(test_df)

train_dataset = Dataset.from_pandas(train_df)
dev_dataset = Dataset.from_pandas(dev_df)
test_dataset = Dataset.from_pandas(test_df)

tokenizer = DistilBertTokenizerFast.from_pretrained(model_name)

def tokenize(batch):
    return tokenizer(batch['text'], padding='max_length', truncation=True, max_length=128)

train_dataset = train_dataset.map(tokenize, batched=True)
dev_dataset = dev_dataset.map(tokenize, batched=True)
test_dataset = test_dataset.map(tokenize, batched=True)

model = DistilBertForSequenceClassification.from_pretrained(
    model_name,
    num_labels=NUM_LABELS,
    problem_type="multi_label_classification",
)

def compute_metrics(pred):
    logits = pred.predictions
    labels = pred.label_ids
    probs = 1 / (1 + np.exp(-logits))
    predictions = (probs > 0.5).astype(int)
    f1_micro = f1_score(labels.flatten(), predictions.flatten(), average='micro')
    return {"f1_micro": f1_micro}

training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="f1_micro",
    greater_is_better=True,
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=dev_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()
trainer.evaluate(eval_dataset=test_dataset)

zero_shot_classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

def tokenize_node(state):
    inputs = tokenizer(state["text"], return_tensors="pt", truncation=True, padding=True, max_length=128)
    state["inputs"] = {k: v.cpu().numpy().tolist() for k, v in inputs.items()}
    return state

def predict_node(state):
    inputs = {k: torch.tensor(v).to(model.device) for k, v in state["inputs"].items()}
    model.eval()
    with torch.no_grad():
        outputs = model(**inputs)
        probs = torch.sigmoid(outputs.logits).squeeze().cpu().numpy()
    state["probs"] = probs.tolist()
    return state

def fallback_node(state):
    probs = np.array(state["probs"])
    top_idx = probs.argmax()
    top_conf = probs[top_idx]
    state["confidence"] = float(top_conf)

    if top_conf < 0.5:
        result = zero_shot_classifier(state["text"], emotion_labels)
        top_emotion = result["labels"][0]
        state["emotion"] = top_emotion
        state["source"] = "backup"
    else:
        top_emotion = emotion_labels[top_idx]
        state["emotion"] = top_emotion
        state["source"] = "primary"
    return state

def save_node(state):
    if state.get("source") == "backup":
        label_idx = emotion_labels.index(state["emotion"])
        new_row = pd.DataFrame([[state["text"], label_idx]])
        df = pd.read_csv(TEMP_DATA_FILE, sep="\t", header=None) if os.path.exists(TEMP_DATA_FILE) else pd.DataFrame()
        df = pd.concat([df, new_row], ignore_index=True)
        df.to_csv(TEMP_DATA_FILE, sep="\t", index=False, header=False)
    return state

def check_and_retrain_node(state):
    if not os.path.exists(TEMP_DATA_FILE): return state

    df = pd.read_csv(TEMP_DATA_FILE, sep="\t", header=None)
    if len(df) >= 10:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        df.to_csv(f"backup_{timestamp}.tsv", sep="\t", index=False, header=False)

        main_df = pd.read_csv("/content/train.tsv", sep="\t", header=None)
        updated_df = pd.concat([main_df, df], ignore_index=True)
        updated_df.to_csv("/content/train.tsv", sep="\t", index=False, header=False)
        os.remove(TEMP_DATA_FILE)

        global train_df, train_dataset
        train_df = convert_to_multihot(updated_df)
        train_dataset = Dataset.from_pandas(train_df)
        train_dataset = train_dataset.map(lambda x: tokenizer(x['text'], padding='max_length', truncation=True, max_length=128), batched=True)

        trainer.train_dataset = train_dataset
        trainer.train()
    return state

def output_node(state):
    text = state.get("text", "")
    emotion = state.get("emotion", "unknown")
    source = state.get("source", "N/A")
    confidence = state.get("confidence", "N/A")

    # Log full structured info
    logging.info(f"{text}\t{emotion}\t{confidence:.2f}\t{source}")
    return state

memory = MemorySaver()
dag = StateGraph(dict)
dag.add_node("Tokenize", tokenize_node)
dag.add_node("Predict", predict_node)
dag.add_node("Fallback", fallback_node)
dag.add_node("Save", save_node)
dag.add_node("CheckRetrain", check_and_retrain_node)
dag.add_node("Output", output_node)

dag.set_entry_point("Tokenize")
dag.add_edge("Tokenize", "Predict")
dag.add_edge("Predict", "Fallback")
dag.add_edge("Fallback", "Save")
dag.add_edge("Save", "CheckRetrain")
dag.add_edge("CheckRetrain", "Output")
dag.add_edge("Output", END)

graph = dag.compile()

# Interactive input loop
while True:
    user_input = input("\nEnter a sentence (or type 'exit' to quit): ").strip()
    if user_input.lower() == 'exit':
        print("Exiting...")
        break
    result = graph.invoke({"text": user_input})
    emotion = result.get("emotion", "Unknown")
    source = result.get("source", "Unknown")
    print(f"\n[User Response]\nText: {user_input}\nPredicted Emotion: {emotion} (via {source})")


/usr/local/lib/python3.11/dist-packages/datasets/table.py:761: UserWarning: The DataFrame has column names of mixed type. They will be converted to strings and not roundtrip correctly.
  return cls(pa.Table.from_pandas(*args, **kwargs))
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

...

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0



Enter a sentence (or type 'exit' to quit): i am happy today

[User Response]
Text: i am happy today
Predicted Emotion: joy (via primary)

Enter a sentence (or type 'exit' to quit): Well, that went exactly how I expected... not.

[User Response]
Text: Well, that went exactly how I expected... not.
Predicted Emotion: disappointment (via backup)

Enter a sentence (or type 'exit' to quit): Cool story, bro.

[User Response]
Text: Cool story, bro.
Predicted Emotion: admiration (via primary)

Enter a sentence (or type 'exit' to quit): I’m here, I guess.

[User Response]
Text: I’m here, I guess.
Predicted Emotion: neutral (via primary)

Enter a sentence (or type 'exit' to quit): exit
Exiting...


In [ ]:
df = pd.read_csv("classification_log.tsv", sep="\t", header=None)
df.columns = ["Timestamp", "Text", "Emotion", "Confidence", "Source"]
df.tail()
